In [1]:
import pandas as pd

# Read in data from "births.csv" and "score.csv" as dataframes
births_df=pd.read_csv("files/births.csv", sep=",")
score_df=pd.read_csv("files/score.csv", sep=",")

In [2]:
# Examine the births dataset
print(births_df.head(10))
print(births_df.dtypes)
print(births_df.describe())

   id  bweight  lowbw    gestwks  preterm  matage  hyp  sex
0   1     2974      0  38.520000      0.0      34    0    2
1   2     3270      0        NaN      NaN      30    0    1
2   3     2620      0  38.150002      0.0      35    0    2
3   4     3751      0  39.799999      0.0      31    0    1
4   5     3200      0  38.889999      0.0      33    1    1
5   6     3673      0  40.970001      0.0      33    0    2
6   7     3628      0  42.139999      0.0      29    0    2
7   8     3773      0  40.209999      0.0      37    0    1
8   9     3960      0  42.029999      0.0      36    0    2
9  10     3405      0  39.330002      0.0      39    0    1
id           int64
bweight      int64
lowbw        int64
gestwks    float64
preterm    float64
matage       int64
hyp          int64
sex          int64
dtype: object
               id      bweight       lowbw     gestwks     preterm  \
count  500.000000   500.000000  500.000000  490.000000  490.000000   
mean   250.500000  3136.884000    

In [3]:
# Transform categorical variables to the right format with labels.
births_df["lowbw"] = births_df["lowbw"].astype("category")
births_df["preterm"] = births_df["preterm"].astype("category")
births_df["hyp"] = births_df["hyp"].astype("category")
births_df["sex"] = births_df["sex"].astype("category")

# Define category renaming mappings.
lowbw_categories = {0: "Normal birthweight", 1: "Low birthweight"}
preterm_categories = {0: "Not preterm", 1: "Preterm"}
hyp_categories = {0: "No maternal hypertension", 1: "Maternal hypertension"}
sex_categories = {1: "Male", 2: "Female"}

# Rename categories for readability.
births_df["lowbw"].cat.rename_categories(lowbw_categories)
births_df["preterm"].cat.rename_categories(preterm_categories)
births_df["hyp"].cat.rename_categories(hyp_categories)
births_df["sex"].cat.rename_categories(sex_categories)

# Display the first few rows of the DataFrame to check the changes.
print(births_df.head())


   id  bweight lowbw    gestwks preterm  matage hyp sex
0   1     2974     0  38.520000     0.0      34   0   2
1   2     3270     0        NaN     NaN      30   0   1
2   3     2620     0  38.150002     0.0      35   0   2
3   4     3751     0  39.799999     0.0      31   0   1
4   5     3200     0  38.889999     0.0      33   1   1


In [4]:
# Create a new DataFrame with any rows with missing data excluded.
births_df_clean = births_df.dropna()
score_df_clean = score_df.dropna()

NameError: name 'scores_df' is not defined

In [ ]:
# Merge two dataframes, keeping matching and unmatching rows
merged_df=pd.merge(births_df_clean, score_df_clean, on="id", how="outer")
# This creates a new datafarme merging by id number. 
merged_df.info()
# The result of this operation includes 490 rows. However only 425 of these have a value in the column "score".



In [ ]:
# Reorder the merged dataset by variable score
sorted_merged_df=merged_df.sort_values(by="score", ascending=True).copy()



In [ ]:

continuous_variables=merged_df[["bweight","matage","gestwks","score"]]
correlation_matrix=continuous_variables.corr()
print("Correlation matrix: ")
print(correlation_matrix)

# This shows a correlation matrix between the continuous variable showing the Pearson correlation coefficient between the variables. 
# Birthweight and gestiational weeks have a moderately strong correlation with a Pearson correlaiton coefficient score of 0.712.




In [ ]:
# Examine the two way distribution of the categorical variables in tabular form.

# Crosstab of sex and hypertension
sex_hyp_group = pd.crosstab(merged_df["sex"], merged_df["hyp"])
print('Sex and Hypertension distribution:')
print(sex_hyp_group)

# Crosstab of sex and preterm births
sex_preterm_group = pd.crosstab(merged_df["sex"], merged_df["preterm"])
print('Sex and Preterm distribution:')
print(sex_preterm_group)

# Crosstab of preterm births and hypertension
preterm_hyp_group = pd.crosstab(merged_df["preterm"], merged_df["hyp"])
print('Preterm and Hypertension distribution:')
print(preterm_hyp_group)

# Crosstab of low birthweight and sex
lowbw_sex_group = pd.crosstab(merged_df["sex"], merged_df["lowbw"])
print('Low birthweight and Sex distribution:')
print(lowbw_sex_group)

# Crosstab of low birthweight and hypertension
lowbw_hyp_group = pd.crosstab(merged_df["lowbw"], merged_df["hyp"])
print('Low birthweight and Hypertension distribution:')
print(lowbw_hyp_group)

# Crosstab of low birthweight and preterm births
lowbw_preterm_group = pd.crosstab(merged_df["lowbw"], merged_df["preterm"])
print('Low birthweight and Preterm distribution:')
print(lowbw_preterm_group)

In [ ]:
import numpy as np

# Create a new variable highscore that identifies a score higher than 150

conditions = [
    merged_df["score"] > 150,  # Condition for scores greater than 150
    merged_df["score"] <= 150,  # Condition for scores less than or equal to 150
    merged_df["score"].isna()   # Condition for NaN scores
]
choices = ["Yes", "No", "No Score"]  # Corresponding choices for the conditions

# Create the 'highscore' column using np.select
merged_df["highscore"] = np.select(conditions, choices, default="No Score")


In [ ]:
# Create an aggregated version of the dataset that reports average birthweight by high score and sex

aggregated_df=merged_df.groupby(["highscore","sex"])["bweight"].agg(average_bweight="mean").reset_index()
print(aggregated_df)


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
sns.regplot(x=merged_df["gestwks"], y=merged_df["bweight"],scatter_kws={"alpha": 0.5})
plt.xlabel("Gestational age (weeks)")
plt.ylabel("Birthweight (grams)")
plt.title("Relationship Between Gestational Weeks and Birthweight", pad=30)
plt.grid(True, linestyle=":")
plt.savefig("Bweightvsgestwks.pdf", format="pdf")
plt.show()




In [ ]:
merged_df.to_excel("cleaned_merged_dataset.xlsx", index=False)